In [ ]:
import re
# import streamlit as st
import time
import torch

from src.sst import download_wav_file, extract_text, load_model
from src.qna import answer_ques

from farm.infer import Inferencer

from ipywidgets import interact, interactive, fixed, interact_manual,widgets
from IPython.display import display, YouTubeVideo

In [64]:
def qna(text,question,nlp):
    answer = answer_ques(text, question, nlp)
    return answer

In [72]:
model_name="deepset/roberta-base-squad2"
display("Loading the QnA model")

device = "gpu" if torch.cuda.is_available() else "cpu"
if device == "gpu":
    nlp = Inferencer.load(model_name, task_type="question_answering", gpu=True)
else:
    nlp = Inferencer.load(model_name, task_type="question_answering", gpu=False)

def extract(title):
  name = re.sub(r'[^\w\s]','_',title)+".wav"
  model, decoder, utils, device = load_model()
  text = extract_text(model, decoder, utils, device, video_url=title, video_name=name)
  return text


'Loading the QnA model'

01/05/2021 05:19:28 - INFO - farm.utils -   Using device: CPU 
01/05/2021 05:19:28 - INFO - farm.utils -   Number of GPUs: 0
01/05/2021 05:19:28 - INFO - farm.utils -   Distributed Training: False
01/05/2021 05:19:28 - INFO - farm.utils -   Automatic Mixed Precision: None
01/05/2021 05:19:28 - INFO - farm.modeling.language_model -   
01/05/2021 05:19:28 - INFO - farm.modeling.language_model -   LOADING MODEL
01/05/2021 05:19:28 - INFO - farm.modeling.language_model -   =============
01/05/2021 05:19:28 - INFO - farm.modeling.language_model -   Could not find deepset/roberta-base-squad2 locally.
01/05/2021 05:19:28 - INFO - farm.modeling.language_model -   Looking on Transformers Model Hub (in local cache and online)...
Some weights of RobertaModel were not initialized from the model checkpoint at deepset/roberta-base-squad2 and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use

In [50]:
def filter_answer(answer=[]):
  if answer[0]['score']>5:
    answer_prob = []
    for ans in answer:
      if ans['score'] >=5:
        answer_prob.append(ans['answer'])
    return answer_prob
  else:
    return answer[0]['answer']

In [73]:
inp = widgets.Text(description='YOutube link:')
button = widgets.Button(description="Click Me!")
Box = widgets.HBox([inp,button])

Text = []

def on_button_clicked(b):
    # value_list.append(inp.value)
    print("Extracting the text from YOutube:")
    try:
      display(YouTubeVideo(inp.value))
    except:
      display(print("The link is broken"))
      return 0
    display("Text extraction on progress")
    extracted_text = extract(inp.value)
    display("Text extraction Complete")
    Text.append(extracted_text)
    return Text

display(Box)
button.on_click(on_button_clicked)

Extracting the text from YOutube:


'Text extraction on progress'

Using cache found in /root/.cache/torch/hub/snakers4_silero-models_master


'Text extraction Complete'

In [76]:
inp2 = widgets.Text(description='Question')
button2 = widgets.Button(description="Answer")
Box2 = widgets.HBox([inp2,button2])
def question(b):
    # value_list.append(inp.value)
    # display(inp2.value)
    # display(Text[-1])
    ans = qna(Text[-1], inp2.value, nlp)
    filtered_ans = filter_answer(ans)
    display("Predicted Answer",filtered_ans)
    return None

display(Box2)
button2.on_click(question)

Inferencing Samples: 100%|██████████| 5/5 [00:16<00:00,  3.28s/ Batches]

biden team
no_answer
president trump
president trump
republicans


'Predicted Answer'

['biden team']